In [17]:
using LinearAlgebra
using NBInclude
@nbinclude("QOJulia_utils.ipynb")

get_basis_mb (generic function with 1 method)

In [6]:
function calc_link_var(ψ1, ψ2)
    link_var = det(ψ1'*ψ2)
    return link_var/abs(link_var)
end

calc_link_var (generic function with 1 method)

In [7]:
function calc_link_vars(UUA)
    N1,N2,__ = size(UUA)
    UU=zeros(Complex{Float64}, N1, N2, 2)
    dirs = [ [1, 0], [0,1] ]
    for it1 in 1:N1
        for it2 in 1:N2
            for (idir, vdir) in enumerate(dirs)
                UU[it1,it2,idir] = calc_link_var(UUA[it1,it2,:,:],
                                                 UUA[mod(it1-1+vdir[1],N1)+1,mod(it2-1+vdir[2],N2)+1,:,:])
            end
        end
    end
    return UU 
end

calc_link_vars (generic function with 1 method)

In [8]:
function calc_F12(UU)
    F12 = log.(  UU[:,:,1].*
                circshift(UU[:,:,2],(-1,0))./
               (circshift(UU[:,:,1],(0,-1)).*
                UU[:,:,2]) 
            )
    return F12
end

calc_F12 (generic function with 1 method)

In [ ]:
function calc_chern_bands(UUA, band)
    Chern = []
    for band in bands
        UU = calc_link_vars(UUA[:,:,:,band[1]:band[2]])
        F12 = calc_F12(UU)
        push!(Chern, sum(F12)/(2im*pi))
    end
    return Chern
end

In [12]:
function calc_all_states_H1(param_dict)
    
    model_H1 = param_dict["model_H1"]
    basis = param_dict["basis"]
    Tx = param_dict[param_dict["param_1"]]
    Ty = param_dict[param_dict["param_2"]]
    dimH = length(basis)
    N1 = length(Tx)
    N2 = length(Ty)
    EEA=zeros(N1, N2, dimH)
    UUA=zeros(Complex{Float64}, N1, N2, dimH, dimH)
    for it1 in 1:N1
        for it2 in 1:N2
            param_dict["θθ"]=[ Tx[it1], Ty[it2] ]
            H1_m = model_H1(param_dict)    
            # H1 = get_sp_op(basis, H1_m)
            EEA[it1,it2,:], UUA[it1,it2,:,:] = eigen(H1_m)
        end
    end
    if "sub_range" in keys(param_dict)
        return EEA[:,:,param_dict["sub_range"]], UUA[:,:,:,param_dict["sub_range"]]
    else
        return EEA, UUA
    end
end

calc_all_states_H1 (generic function with 1 method)

In [15]:
function calc_all_states_H_mb(param_dict)
    
    Tx = param_dict[param_dict["param_1"]]
    Ty = param_dict[param_dict["param_2"]]
    dimH = param_dict["dimH_mb"]
    N1 = length(Tx)
    N2 = length(Ty)
    EEA=zeros(N1, N2, dimH)
    UUA=zeros(Complex{Float64}, N1, N2, dimH, dimH)
    for it1 in 1:N1
        for it2 in 1:N2
            param_dict["θθ"]=[ Tx[it1], Ty[it2] ]
            H_mb = get_H_mb(param_dict)
            EEA[it1,it2,:], UUA[it1,it2,:,:] = eigen(dense(H_mb).data)
        end
    end

    return EEA, UUA
end

calc_all_states_Hmb (generic function with 1 method)